In [1]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,"../")
import mplhep as hep
import pickle
import glob
import ROOT as rt
import coffea
import awkward as ak
from coffea import hist, processor
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
import os
import json

Warning in <TInterpreter::ReadRootmapFile>: class  HepMC::FourVector found in libSimDataFormatsGeneratorProducts.so  is already in libHepMC3rootIO.so 


Welcome to JupyROOT 6.28/00


/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  r

In [17]:
rt.gStyle.SetOptStat(0)
def make_ratio_plot(campaign, h_list_in, title = "", label = "", fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1], return_dataMC = False):
    h_list = []
    if in_tags == None:
        tag = []
    else:
        tag = in_tags
    for i, h in enumerate(h_list_in):
        h_cloned = h.Clone('h{}aux{}'.format(i, label))
        #if i==0: h_cloned.Scale(2091.7/2219) #UNCOMMENT FOR CLUSTER PLOTS, K-FACTOR TAKES CARE OF IT
        print(h_cloned.GetSumOfWeights())
        h_list.append(h_cloned)
        if in_tags == None:
            tag.append(h.GetTitle())
    #print("tags: ", tag)
    global c_out
    c_out = rt.TCanvas("c_out_ratio", "c_out_ratio", 800, 800)
    pad1 = rt.TPad("pad1a", "pad1a", 0, 0.3, 1, 1.0)
    pad1.SetBottomMargin(0.03)
    pad1.SetLeftMargin(0.15)
    pad1.SetRightMargin(0.04)# pad2.SetGrid()
    if logy:
        pad1.SetLogy()

    pad1.Draw()
    pad1.cd()

    leg = rt.TLegend(0.5, 0.65, 0.9, 0.92)
    leg = rt.TLegend(0.8, 0.75, 0.93, 0.9)

    #leg = rt.TLegend(0.2, 0.7, 0.5, 0.9)
    # leg = rt.TLegend(0.7, 0.2, 0.9, 0.4)
    leg.SetBorderSize(0)
    leg.SetTextSize(0.045)
    leg.SetFillStyle(0)
    c_out.cd(1)

    scaled_h_list = []
    if scale:
        for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #h = h_unscaled.Scale(1/scales[i])
            #scaled_h_list.append(h_unscaled.Clone())
            h_unscaled.Scale(1/scales[i])
            scaled_h_list.append(h_unscaled)
    else:
        #for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #scaled_h_list.append(h)
        scaled_h_list = h_list
    for i, h in enumerate(scaled_h_list):
        h.GetXaxis().SetLabelSize(0)
        h.GetXaxis().SetTitle(label)
        h.GetYaxis().SetRangeUser(0, 1.1*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and not scale:
            h.GetYaxis().SetRangeUser(10e-2, 2*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and scale:
            h.GetYaxis().SetRangeUser(10e-4, 1)
        h.GetYaxis().SetTitleOffset(1.0)
        h.GetYaxis().SetTitleSize(0.06)
        h.GetYaxis().SetLabelSize(0.05)
        
        if scale:
            y_title = "Fraction of Events"
        else:
            y_title = "Events"
        
        h.GetYaxis().SetTitle()
        h.SetTitle(f";{label};{y_title}")
        #if ratio_index == 0:h.DrawCopy("hist")
        '''
        h.SetFillColor(h_list_in[i].GetLineColor())
        h.SetFillStyle(3002)
        #h.SetStats(1)
        h.SetLineColor(h_list_in[i].GetLineColor())
        h.SetLineWidth(2)
        h.SetMarkerColor(h_list_in[i].GetLineColor())
        h.SetMarkerSize(2)
        # if ratio_index == 0:
        #     # h.DrawCopy("hist")
        #     h.DrawCopy(draw_opt[i]+'same')
        # else:h.DrawCopy(draw_opt[i])
        #if ratio_index == 0 :h.DrawCopy(draw_opt[i]+"same")
        #h.DrawCopy("E2 HIST")
        '''
        if i==0:
            h.SetLineWidth(4)
            h.DrawCopy("hist")
            #h.SetFillStyle(0)
            h.SetFillColor(h_list_in[i].GetLineColor())
            h.SetFillStyle(3002)
            #h.SetStats(1)
            h.SetLineColor(h_list_in[i].GetLineColor())
            h.SetLineWidth(2)
            h.SetMarkerColor(h_list_in[i].GetLineColor())
            h.SetMarkerSize(2)
            h.DrawCopy(draw_opt[i] + "same")
            #h.SetFillStyle(0)
        else:
            h.SetLineWidth(2)
            h.DrawCopy(draw_opt[i] + "same")
        #else:h.DrawCopy(draw_opt[i])
        if len(text)>0:
            l = rt.TLatex()
            l.SetTextSize(0.045)
            if logy:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()/10, text)
            else:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()*0.8, text)
        #if i==1:
            #h.DrawCopy(draw_opt[i]+"same")
       #     h.Draw("E1 same")
        if i==1:
            leg.AddEntry(h, tag[i], "lep")
        else:
            leg.AddEntry(h, tag[i], "l")
    leg.Draw("same")
    cmsText = rt.TLatex()

    cmsText.SetNDC(True)

    #labelStr = "CMS Preliminary " + campaignStr
    cmsText.SetTextFont(42)
    cmsText.SetTextSize(0.045)
    cmsText.SetTextAlign(11)
    if campaign=="2022":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2022} (7.98 fb^{-1})")
    if campaign=="2022EE":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2022} (26.67 fb^{-1})")
    if campaign=="2023":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2023} (18.41 fb^{-1})")
    if campaign=="2023BPix":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2023} (9.45 fb^{-1})")
    if campaign=="2024":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2024} (109.08 fb^{-1})")
    if campaign=="all":
        cmsText.DrawLatex(0.33, 0.91, "#bf{CMS Preliminary 2022, 2023, 2024} (171.59 fb^{-1})")

    scaled_h_list_copy = [scaled_h_list[0].Clone(), scaled_h_list[1].Clone()]
    c_out.cd()
    pad2 = rt.TPad("pad2a", "pad2a", 0, 0, 1, 0.3)
    pad2.SetTopMargin(0.03)
    pad2.SetBottomMargin(0.25)
    pad2.SetLeftMargin(0.15)
    pad2.SetRightMargin(0.04)# pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    band = scaled_h_list[ratio_index].Clone('h_band')
    for j in range(band.GetXaxis().GetNbins()):
        band.SetBinContent(j+1, 1.0)
        if h_list[ratio_index].GetBinContent(j+1) == 0:
            band.SetBinError(j+1, 0.0)
        else:
            band.SetBinError(j+1, scaled_h_list[ratio_index].GetBinError(j+1)/scaled_h_list[ratio_index].GetBinContent(j+1))
            #print(j, h_list_in[0].GetBinError(j+1)/h_list_in[0].GetBinContent(j+1))
    band.SetFillColor(scaled_h_list[ratio_index].GetLineColor())

    band.SetFillStyle(3002)
    band.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    #band.SetFillColorAlpha(0,0)
    #band.SetLineColor(0)
    
    band.GetYaxis().SetTitleOffset(0.5)
    band.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
    band.GetYaxis().SetTitleSize(0.11)
    band.GetYaxis().SetLabelSize(0.12)
    band.GetYaxis().SetNdivisions(506)
    band.GetXaxis().SetTitleOffset(0.95)
    band.GetXaxis().SetTitleSize(0.12)
    band.GetXaxis().SetLabelSize(0.12)
    band.GetXaxis().SetTickSize(0.07)
    
    band.SetYTitle('Data / MC'.format(tag[ratio_index]))
    band.SetXTitle(label)
    band.SetTitle("")
    band.DrawCopy('E2')
    ln = rt.TLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
    ln.SetLineWidth(3)
    ln.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    ln.DrawLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
     
    #print(ratio_index)
    for i, h in enumerate(scaled_h_list):
        if i == ratio_index:
            continue
        else:
            if fit:h.GetFunction("expo")
            h.Divide(scaled_h_list[ratio_index])
            # h.GetYaxis().SetTitleOffset(0.6)
            h.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
            # h.GetYaxis().SetTitleSize(0.12)
            # h.GetYaxis().SetLabelSize(0.12)
            # h.GetYaxis().SetNdivisions(506)
            # h.GetXaxis().SetTitleOffset(0.95)
            # h.GetXaxis().SetTitleSize(0.12)
            # h.GetXaxis().SetLabelSize(0.12)
            # h.GetXaxis().SetTickSize(0.07)
            # h.SetYTitle('Ratio with {}'.format(tag[0]))
            # h.SetTitle("")
            #set relative error of ratio to be the relative error of data
            for j in range(h.GetXaxis().GetNbins()):
                if h_list[i].GetBinContent(j+1) == 0:
                    h.SetBinError(j+1, 0.0)
                else:
                    h.SetBinError(j+1, h_list_in[i].GetBinError(j+1)/h_list_in[i].GetBinContent(j+1)*h.GetBinContent(j+1))
            h.Draw('same'+draw_opt[i])
    
    pad2.Update()
    
    if return_dataMC:
        return h
    
    c_out.pad1 = pad1
    c_out.pad2 = pad2
    c_out.h_list = h_list
    c_out.leg = leg
    
    
    return c_out, scaled_h_list_copy, scaled_h_list[1]

In [18]:
campaigns = ["2022","2022EE","2023","2023BPix","2024"]

## No Clusters

In [14]:
noClusters_filepath = "_Data_MC_Comp_noClusters_correctSFs"
plot_info = {"ZMass": {"inputName":"ZMass", "title":"Dimuon Mass Distribution", 
                       "xlabel":"Dimuon Mass [GeV]", "bins":(80, 40, 130), "logy":False}}

In [19]:
plot_output = f"CombinedCampaigns_Data_MC_Comp_noClusters"
os.makedirs(plot_output, exist_ok=True)
for plotName, plot_info_dict in plot_info.items():
    #if plotName=="clusterSize":continue
    print(plotName)
    for idx, campaign in enumerate(campaigns):
        inFile = rt.TFile.Open(campaign+noClusters_filepath+"/"+plot_info_dict["inputName"]+"/"+plot_info_dict["inputName"]+".root", "READ")
        if idx==0:
            Data_hist = inFile.Get("Data_hist")
            MC_hist = inFile.Get("MC_hist")
            Data_hist_total = Data_hist.Clone()
            MC_hist_total = MC_hist.Clone()
            Data_hist_total.SetDirectory(0)
            MC_hist_total.SetDirectory(0)

        else:
            Data_hist = inFile.Get("Data_hist")
            Data_hist  = Data_hist.Clone()
            Data_hist_total.Add(Data_hist)
            MC_hist = inFile.Get("MC_hist")
            MC_hist  = MC_hist.Clone()
            MC_hist_total.Add(MC_hist)
            
        inFile.Close()
        Data_hist_total.SetTitle("Data")
        Data_hist_total.SetFillStyle(0)
        MC_hist_total.SetTitle("MC")
        MC_hist_total.SetFillStyle(0)
        
    #plot_info[plotName]["hists"] = [MC_hist_total, Data_hist_total]
    c, h_list, ratio  = make_ratio_plot("all", [MC_hist_total, Data_hist_total], title = plot_info_dict["title"], label = plot_info_dict["xlabel"], fit = False, in_tags = None, ratio_bounds = [0.5, 2], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1])
    os.makedirs(plot_output, exist_ok=True)
    c.SaveAs(plot_output+"/"+plot_info_dict["inputName"]+".png")
    file = rt.TFile(plot_output+"/"+plot_info_dict["inputName"]+".root", "RECREATE")
    #file.cd()
    c.Write()
    h_list[0].Write("MC_hist")
    h_list[1].Write("Data_hist")
    ratio.Write("Data_MC_Ratio")
    file.Close()

ZMass
203632629.93359375
192384458.0


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_noClusters/ZMass.png has been created


## CSCs

#### low MET

In [3]:
CSC_filepath = "_Data_MC_Comp_finalzedSelections_fixedHotspotTrue_correctSFs"
plot_info  = {"clusterSize": {"inputName": "cscRechitClusterSize_normToDataCounts", "title":"Cluster Size Distribution", 
                       "xlabel":"N_{hits}","bins":(60, 0, 600), "logy":True},
              "forward_hits": {"inputName":"forward_hits_normToDataCounts", "title":"NHits ME11/12", 
                       "xlabel":"N_{hits}","bins":(61, -1, 60), "logy":True},
              "cscRechitClusterDNN_bkgMC_plusBeamHalo": {"inputName":"cscRechitClusterDNN_bkgMC_plusBeamHalo_normToDataCounts", "title":"DNN Score Distribution", 
                       "xlabel":"DNN Score","bins":(12, 0.52, 1), "logy":True},
              "cscRechitClusterTimeWeighted": {"inputName":"cscRechitClusterTimeWeighted_normToDataCounts", "title":"Weighted Cluster Time Distribution", 
                       "xlabel":"Weighted Cluster Time [ns]","bins":(60, -8, 20), "logy":False},
             "cscRechitClusterTimeSpreadWeightedAll": {"inputName":"cscRechitClusterTimeSpreadWeightedAll_normToDataCounts", "title":"Weighted Cluster Timespread Distribution", 
                       "xlabel":"Weighted Cluster Timespread [ns]", "bins":(60, 0, 50),"logy":False}, 
            "cscRechitClusterNStation10": {"inputName":"cscRechitClusterNStation10_normToDataCounts", "title":"Number of Stations with >=10 Rechits", 
                       "xlabel":"# of Stations","bins":(7, -1, 6), "logy":False}
             }

In [4]:
clusterSize_bins = np.concatenate((np.linspace(0,160, 17), np.linspace(200, 560, 10)))
clusterSize_bins_highMET = np.concatenate((np.linspace(0,150, 16), np.linspace(190, 550, 10)))

In [5]:
plot_output = f"CombinedCampaigns_Data_MC_Comp_lowMETCSCs"
os.makedirs(plot_output, exist_ok=True)
for plotName, plot_info_dict in plot_info.items():
    #if plotName=="clusterSize":continue
    print(plotName)
    for idx, campaign in enumerate(campaigns):
        inFile = rt.TFile.Open(campaign+CSC_filepath+"/"+plot_info_dict["inputName"]+"/"+plot_info_dict["inputName"]+".root", "READ")
        if idx==0:
            Data_hist = inFile.Get("Data_hist")
            MC_hist = inFile.Get("MC_hist")
            Data_hist_total = Data_hist.Clone()
            MC_hist_total = MC_hist.Clone()
            Data_hist_total.SetDirectory(0)
            MC_hist_total.SetDirectory(0)

        else:
            Data_hist = inFile.Get("Data_hist")
            Data_hist  = Data_hist.Clone()
            Data_hist_total.Add(Data_hist)
            MC_hist = inFile.Get("MC_hist")
            MC_hist  = MC_hist.Clone()
            MC_hist_total.Add(MC_hist)
            
        inFile.Close()
        Data_hist_total.SetTitle("Data")
        Data_hist_total.SetFillStyle(0)
        MC_hist_total.SetTitle("MC")
        MC_hist_total.SetFillStyle(0)
        
    #plot_info[plotName]["hists"] = [MC_hist_total, Data_hist_total]
    c, h_list, ratio  = make_ratio_plot("all", [MC_hist_total, Data_hist_total], title = plot_info_dict["title"], label = plot_info_dict["xlabel"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1])
    os.makedirs(plot_output, exist_ok=True)
    c.SaveAs(plot_output+"/"+plot_info_dict["inputName"]+".png")
    file = rt.TFile(plot_output+"/"+plot_info_dict["inputName"]+".root", "RECREATE")
    #file.cd()
    c.Write()
    h_list[0].Write("MC_hist")
    h_list[1].Write("Data_hist")
    ratio.Write("Data_MC_Ratio")
    file.Close()

clusterSize
forward_hits
cscRechitClusterDNN_bkgMC_plusBeamHalo
cscRechitClusterTimeWeighted
cscRechitClusterTimeSpreadWeightedAll
cscRechitClusterNStation10


Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETCSCs/cscRechitClusterSize_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETCSCs/forward_hits_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETCSCs/cscRechitClusterDNN_bkgMC_plusBeamHalo_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETCSCs/cscRechitClusterTimeWeighted_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETCSCs/cscRechitClusterTimeSpreadWeightedAll_normToD

#### high MET


In [7]:
CSC_filepath = "_Data_MC_Comp_finalzedSelections_highMET_fixedHotspotTrue_correctSFs"
plot_info  = {"clusterSize": {"inputName": "cscRechitClusterSize_normToDataCounts", "title":"Cluster Size Distribution", 
                       "xlabel":"N_{hits}","bins":(60, 0, 600), "logy":True},
              "forward_hits": {"inputName":"forward_hits_normToDataCounts", "title":"NHits ME11/12", 
                       "xlabel":"N_{hits}","bins":(61, -1, 60), "logy":True},
              "cscRechitClusterTimeWeighted": {"inputName":"cscRechitClusterTimeWeighted_normToDataCounts", "title":"Weighted Cluster Time Distribution", 
                       "xlabel":"Weighted Cluster Time [ns]","bins":(60, -8, 20), "logy":False},
             }

In [8]:
plot_output = f"CombinedCampaigns_Data_MC_Comp_highMETCSCs"
os.makedirs(plot_output, exist_ok=True)
for plotName, plot_info_dict in plot_info.items():
    #if plotName=="clusterSize":continue
    print(plotName)
    for idx, campaign in enumerate(campaigns):
        inFile = rt.TFile.Open(campaign+CSC_filepath+"/"+plot_info_dict["inputName"]+"/"+plot_info_dict["inputName"]+".root", "READ")
        if idx==0:
            Data_hist = inFile.Get("Data_hist")
            MC_hist = inFile.Get("MC_hist")
            Data_hist_total = Data_hist.Clone()
            MC_hist_total = MC_hist.Clone()
            Data_hist_total.SetDirectory(0)
            MC_hist_total.SetDirectory(0)

        else:
            Data_hist = inFile.Get("Data_hist")
            Data_hist  = Data_hist.Clone()
            Data_hist_total.Add(Data_hist)
            MC_hist = inFile.Get("MC_hist")
            MC_hist  = MC_hist.Clone()
            MC_hist_total.Add(MC_hist)
            
        inFile.Close()
        Data_hist_total.SetTitle("Data")
        Data_hist_total.SetFillStyle(0)
        MC_hist_total.SetTitle("MC")
        MC_hist_total.SetFillStyle(0)
        
    #plot_info[plotName]["hists"] = [MC_hist_total, Data_hist_total]
    c, h_list, ratio  = make_ratio_plot("all", [MC_hist_total, Data_hist_total], title = plot_info_dict["title"], label = plot_info_dict["xlabel"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1])
    os.makedirs(plot_output, exist_ok=True)
    c.SaveAs(plot_output+"/"+plot_info_dict["inputName"]+".png")
    file = rt.TFile(plot_output+"/"+plot_info_dict["inputName"]+".root", "RECREATE")
    #file.cd()
    c.Write()
    h_list[0].Write("MC_hist")
    h_list[1].Write("Data_hist")
    ratio.Write("Data_MC_Ratio")
    file.Close()

clusterSize
forward_hits
cscRechitClusterTimeWeighted


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_highMETCSCs/cscRechitClusterSize_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_highMETCSCs/forward_hits_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_highMETCSCs/cscRechitClusterTimeWeighted_normToDataCounts.png has been created


## DTs

In [8]:
DT_filepath = "_DT_Data_MC_Comp_finalzedSelections_DTs"
plot_info  = {"dtRechitClusterSize": {"inputName":"dtRechitClusterSize_normToDataCounts", "title":"Cluster Size Distribution", 
                       "xlabel":"N_{hits}", "bins":(40, 0, 200), "logy":True},
            "dtRechitCluster_match_RPChits_dPhi0p5": {"inputName":"dtRechitCluster_match_RPChits_dPhi0p5_normToDataCounts", "title":"Number of Matched RPC Hits", 
                       "xlabel":"Matched RPC Hits", "bins":(26, -1, 26), "logy":False},
            "dtRechitCluster_match_RPCBx_dPhi0p5": {"inputName":"dtRechitCluster_match_RPCBx_dPhi0p5_normToDataCounts", "title":"Bunch Crossing with Most Hits", 
                       "xlabel":"Bunch Crossing","bins":(10, -4, 6), "logy":False}
             }

In [9]:
plot_output = f"CombinedCampaigns_Data_MC_Comp_lowMETDTs"
os.makedirs(plot_output, exist_ok=True)
for plotName, plot_info_dict in plot_info.items():
    #if plotName=="clusterSize":continue
    print(plotName)
    for idx, campaign in enumerate(campaigns):
        inFile = rt.TFile.Open(campaign+DT_filepath+"/"+plot_info_dict["inputName"]+"/"+plot_info_dict["inputName"]+".root", "READ")
        if idx==0:
            Data_hist = inFile.Get("Data_hist")
            MC_hist = inFile.Get("MC_hist")
            Data_hist_total = Data_hist.Clone()
            MC_hist_total = MC_hist.Clone()
            Data_hist_total.SetDirectory(0)
            MC_hist_total.SetDirectory(0)

        else:
            Data_hist = inFile.Get("Data_hist")
            Data_hist  = Data_hist.Clone()
            Data_hist_total.Add(Data_hist)
            MC_hist = inFile.Get("MC_hist")
            MC_hist  = MC_hist.Clone()
            MC_hist_total.Add(MC_hist)
            
        inFile.Close()
        Data_hist_total.SetTitle("Data")
        Data_hist_total.SetFillStyle(0)
        MC_hist_total.SetTitle("MC")
        MC_hist_total.SetFillStyle(0)
        
    #plot_info[plotName]["hists"] = [MC_hist_total, Data_hist_total]
    c, h_list, ratio  = make_ratio_plot("all", [MC_hist_total, Data_hist_total], title = plot_info_dict["title"], label = plot_info_dict["xlabel"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1])
    os.makedirs(plot_output, exist_ok=True)
    c.SaveAs(plot_output+"/"+plot_info_dict["inputName"]+".png")
    file = rt.TFile(plot_output+"/"+plot_info_dict["inputName"]+".root", "RECREATE")
    #file.cd()
    c.Write()
    h_list[0].Write("MC_hist")
    h_list[1].Write("Data_hist")
    ratio.Write("Data_MC_Ratio")
    file.Close()

dtRechitClusterSize
dtRechitCluster_match_RPChits_dPhi0p5
dtRechitCluster_match_RPCBx_dPhi0p5


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETDTs/dtRechitClusterSize_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETDTs/dtRechitCluster_match_RPChits_dPhi0p5_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_lowMETDTs/dtRechitCluster_match_RPCBx_dPhi0p5_normToDataCounts.png has been created


#### high MET

In [10]:
DT_filepath = "_DT_Data_MC_Comp_finalzedSelections_highMET_DTs"
plot_info  = {"dtRechitClusterSize": {"inputName":"dtRechitClusterSize_normToDataCounts", "title":"Cluster Size Distribution", 
                       "xlabel":"N_{hits}", "bins":(40, 0, 200), "logy":True},
            "dtRechitCluster_match_RPChits_dPhi0p5": {"inputName":"dtRechitCluster_match_RPChits_dPhi0p5_normToDataCounts", "title":"Number of Matched RPC Hits", 
                       "xlabel":"Matched RPC Hits", "bins":(26, -1, 26), "logy":False},
            "dtRechitCluster_match_RPCBx_dPhi0p5": {"inputName":"dtRechitCluster_match_RPCBx_dPhi0p5_normToDataCounts", "title":"Bunch Crossing with Most Hits", 
                       "xlabel":"Bunch Crossing","bins":(10, -4, 6), "logy":False}
             }

In [11]:
plot_output = f"CombinedCampaigns_Data_MC_Comp_highMETDTs"
os.makedirs(plot_output, exist_ok=True)
for plotName, plot_info_dict in plot_info.items():
    #if plotName=="clusterSize":continue
    print(plotName)
    for idx, campaign in enumerate(campaigns):
        inFile = rt.TFile.Open(campaign+DT_filepath+"/"+plot_info_dict["inputName"]+"/"+plot_info_dict["inputName"]+".root", "READ")
        if idx==0:
            Data_hist = inFile.Get("Data_hist")
            MC_hist = inFile.Get("MC_hist")
            Data_hist_total = Data_hist.Clone()
            MC_hist_total = MC_hist.Clone()
            Data_hist_total.SetDirectory(0)
            MC_hist_total.SetDirectory(0)

        else:
            Data_hist = inFile.Get("Data_hist")
            Data_hist  = Data_hist.Clone()
            Data_hist_total.Add(Data_hist)
            MC_hist = inFile.Get("MC_hist")
            MC_hist  = MC_hist.Clone()
            MC_hist_total.Add(MC_hist)
            
        inFile.Close()
        Data_hist_total.SetTitle("Data")
        Data_hist_total.SetFillStyle(0)
        MC_hist_total.SetTitle("MC")
        MC_hist_total.SetFillStyle(0)
        
    #plot_info[plotName]["hists"] = [MC_hist_total, Data_hist_total]
    c, h_list, ratio  = make_ratio_plot("all", [MC_hist_total, Data_hist_total], title = plot_info_dict["title"], label = plot_info_dict["xlabel"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1])
    os.makedirs(plot_output, exist_ok=True)
    c.SaveAs(plot_output+"/"+plot_info_dict["inputName"]+".png")
    file = rt.TFile(plot_output+"/"+plot_info_dict["inputName"]+".root", "RECREATE")
    #file.cd()
    c.Write()
    h_list[0].Write("MC_hist")
    h_list[1].Write("Data_hist")
    ratio.Write("Data_MC_Ratio")
    file.Close()

dtRechitClusterSize
dtRechitCluster_match_RPChits_dPhi0p5
dtRechitCluster_match_RPCBx_dPhi0p5


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_highMETDTs/dtRechitClusterSize_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_highMETDTs/dtRechitCluster_match_RPChits_dPhi0p5_normToDataCounts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file CombinedCampaigns_Data_MC_Comp_highMETDTs/dtRechitCluster_match_RPCBx_dPhi0p5_normToDataCounts.png has been created
